In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from os.path import join as pjoin

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
HOME  = pjoin(os.getcwd(), "drive", "MyDrive", "Projects", "Face Retrieval")

In [6]:
os.listdir(HOME)

['TransferLearning_FaceRetrieval.ipynb',
 'haarcascade_frontalface_default.xml',
 'lfw-deepfunneled',
 'dataset_lfw',
 'dataset_lfw_cropped',
 'dataset_lfw_pairs',
 'images_pair1.npy',
 'labels_pair1.npy',
 'saved_model',
 'saved_model1',
 'history_of_model',
 'images_pair.npy',
 'labels_pair.npy',
 'min_2.npy',
 'max_1.npy',
 'image_pairs.npy',
 'labels.npy',
 'efficient_saved_model',
 'Face_Retrieval.ipynb']

In [ ]:
# images_pair = np.load(pjoin(HOME,"images_pair.npy"))
# labels_pair = np.load(pjoin(HOME,"labels_pair.npy"))

In [ ]:
# images_pair.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(images_pair, labels_pair, test_size=0.1, shuffle=True, random_state=5)

In [ ]:
# images_pair1 = np.load(pjoin(HOME,"images_pair1.npy"))
# labels_pair1 = np.load(pjoin(HOME,"labels_pair1.npy"))

In [ ]:
# images_pair1.shape

(11424, 2, 114, 114, 3)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(images_pair1, labels_pair1, test_size=0.1, shuffle=True, random_state=5)

In [7]:
image_pairs = np.load((pjoin(HOME,"image_pairs.npy")))
labels = np.load(pjoin(HOME, "labels.npy"))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(image_pairs, labels, test_size=0.1, shuffle=True, random_state=5)

In [10]:
X_train.shape

(13386, 2, 114, 114, 3)

In [11]:
SHAPE = (114, 114, 3)

In [14]:
def get_feature_extraction_block(back_bone="efficient_v2"):
  input = tf.keras.Input(shape=SHAPE)

  if back_bone == "efficient_v2":
    cnn_module = tf.keras.applications.efficientnet_v2.EfficientNetV2L(include_top=False,input_shape=SHAPE)
  elif back_bone == "resnet50":  
    cnn_module = tf.keras.applications.ResNet50(include_top=False,input_shape=SHAPE)
  elif back_bone == "vgg16":
    cnn_module = tf.keras.applications.VGG16(include_top=False,input_shape=SHAPE)

  for layer in cnn_module.layers:
      layer.trainable = False
  features = cnn_module(input)
  features = tf.keras.layers.Flatten()(features)
  embedd = tf.keras.layers.Dense(1024)(features)
  output = tf.keras.layers.Dense(512)(embedd)

  model = tf.keras.Model(inputs=input, outputs=output, name="Feature_Extractor")
  return model

In [12]:
from tensorflow.keras import backend as k

def euclidean_distance(vectors):
  (featA, featB) = vectors
  sum_squared = k.sum(k.square(featA - featB), axis=1, keepdims=True)
  return k.sqrt(k.maximum(sum_squared, k.epsilon()))

In [13]:
def get_siamese_network(back_bone="efficient_v2"):
  inputA = tf.keras.Input(shape=SHAPE)
  inputB = tf.keras.Input(shape=SHAPE)

  feature_extractor = get_feature_extraction_block(back_bone=back_bone)

  featuresA = feature_extractor(inputA)
  featuresB = feature_extractor(inputB)

  distance = tf.keras.layers.Lambda(euclidean_distance)([featuresA, featuresB])
  output = tf.keras.layers.Dense(1, activation="sigmoid")(distance)

  model = tf.keras.Model(inputs=[inputA, inputB], outputs=output)
  return model

In [15]:
model1 = get_siamese_network(back_bone="efficient_v2")
# model2 = get_siamese_network(back_bone="resnet50")
# model3 = get_siamese_network(back_bone="vgg16")

473176280/473176280 [==============================] - 21s 0us/step


In [16]:
model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
class StopOnPoint(tf.keras.callbacks.Callback):
    def __init__(self, point):
        super(StopOnPoint, self).__init__()
        self.point = point

    def on_epoch_end(self, epoch, logs=None): 
        accuracy = logs["accuracy"]
        if accuracy >= self.point:
            self.model.stop_training = True

callbacks = [StopOnPoint(0.98)]

In [ ]:
history1 = model1.fit([X_train[:, 0], X_train[:, 1]], y_train[:],validation_split=0.1,batch_size=64,epochs=100, callbacks=callbacks)

Epoch 1/100
189/189 [==============================] - 176s 653ms/step - loss: 9.3543 - accuracy: 0.4990 - val_loss: 3.8469 - val_accuracy: 0.5019
Epoch 2/100
189/189 [==============================] - 110s 583ms/step - loss: 1.9639 - accuracy: 0.4990 - val_loss: 0.8522 - val_accuracy: 0.5019
Epoch 3/100
189/189 [==============================] - 110s 580ms/step - loss: 0.8518 - accuracy: 0.4990 - val_loss: 0.8464 - val_accuracy: 0.5019
Epoch 4/100
189/189 [==============================] - 110s 580ms/step - loss: 0.9916 - accuracy: 0.4990 - val_loss: 1.0525 - val_accuracy: 0.5019
Epoch 5/100
189/189 [==============================] - 110s 580ms/step - loss: 1.2777 - accuracy: 0.4990 - val_loss: 1.1446 - val_accuracy: 0.5019
Epoch 6/100
189/189 [==============================] - 110s 580ms/step - loss: 1.2960 - accuracy: 0.4992 - val_loss: 1.1406 - val_accuracy: 0.5019
Epoch 7/100
189/189 [==============================] - 110s 580ms/step - loss: 1.0369 - accuracy: 0.5009 - val_loss: 0

In [ ]:
model1.save(pjoin(HOME,"efficientnet_saved_model1"))

In [ ]:
results = model1.evaluate([X_test[:,0], X_test[:,1]], y_test)

In [ ]:
loss = np.array(history1.history['loss'])
accuracy = np.array(history1.history['accuracy'])
val_loss = np.array(history1.history['val_loss'])
val_accuracy = np.array(history1.history['val_accuracy'])

In [ ]:
np.save(pjoin(HOME,"eff_his","loss"),loss)
np.save(pjoin(HOME,"eff_his","accuracy"),accuracy)
np.save(pjoin(HOME,"eff_his","val_loss"),val_loss)
np.save(pjoin(HOME,"eff_his","val_accuracy"),val_accuracy)